# Perceptrons

At this point in the semester, you have learned about the perceptron algorithm and how it finds a linear decision boundary for binary classification. The goal here is to solidify your understanding. In completing this notebook you will become more familiar with some important concepts in machine learning:
- __Learning algorithms__
    - Review the perceptron learning rule
- __The relevance of theory in practice__
    - Review the Perceptron Convergence Theorem
    - See how the theorem might tell us something about model development
- __Interpreting linear models__
    - Understand what a perceptron model is doing by looking at its parameters, and how this can be useful
    - Observe the dependence of perceptron interpretability on the feature inputs
- __Applications__
    - Train a perceptron on sentiment data
    - Evaluate the model
    - Interpret the model

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import runClassifier
import datasets
import matplotlib.pyplot as plt
import sklearn

import perceptron

## The perceptron learning rule

__TASK 1:__ We have provided a partial implementation of the perceptron algorithm in `perceptron.py`, with one thing missing: code for updating the weights and bias. You will write this code. The update happens in the `nextExample` function. Open `perceptron.py` and complete `nextExample` so that it updates the weight vector and bias according to the perceptron rule.

Now let's see whether the weight updates work, with a simple test. We'll train the perceptron on `TennisData` and check the accuracy. The `BinaryClassifier` class will handle training on datasets for you, as long as you specify the number of epochs to run.

- For the first cell, you should get a training accuracy of 0.642857, and a test accuracy of 0.666667.
- For the second cell, you should get a training accuracy of 0.857143, and a test accuracy of 1.

In [ ]:
runClassifier.trainTestSet(perceptron.Perceptron({'numEpoch': 1}), datasets.TennisData)

In [ ]:
runClassifier.trainTestSet(perceptron.Perceptron({'numEpoch': 2}), datasets.TennisData)

Now let's see whether the perceptron finds good decision boundaries. We'll start with a very simple dataset in 2 dimensions: `TwoFarGaussians`. This dataset is essentially two far-away clusters of points in 2-dimensional space (sampled from two different bivariate Gaussian distributions). Points are labeled `+1` in the first cluster, and `-1` in the other.

The clusters are non-overlapping, so the Perceptron Convergence Theorem tells us that the algorithm should converge. The clusters are also far away from each other, so it should be easy to find a line that separates the clusters. Let's train on this data, and plot the resulting decision boundary.

In [ ]:
data = datasets.TwoFarGaussians
h = perceptron.Perceptron({'numEpoch': 2})
h.train(data.X, data.Y)
runClassifier.plotDataAndBoundary(data.X, data.Y, h.weights, h.bias)
h

The perceptron algorithm should find a separating hyperplane.

## Perceptron Convergence Theorem

Let's compare this perceptron with a perceptron trained on a slightly different dataset: `TwoCloseGaussians`. The only significant difference here is that the clusters are now closer together (in more precise terms, each distribution has the same covariance, but the means are now closer together). Let's see our decision boundary after 2 epochs.

In [ ]:
data = datasets.TwoCloseGaussians
h = perceptron.Perceptron({'numEpoch': 2})
h.train(data.X, data.Y)
runClassifier.plotDataAndBoundary(data.X, data.Y, h.weights, h.bias)
h

The perceptron does not converge after 2 epochs. But all we did was move the Gaussian clusters closer together! For some interesting reason, the perceptron needs more exposure to the data.

__TASK 2:__ Change the `numEpoch` hyperparameter below to a small number that allows the perceptron to classify all the points.

In [ ]:
h = perceptron.Perceptron({'numEpoch': 2}) # change the number of epochs
h.train(data.X, data.Y)
runClassifier.plotDataAndBoundary(data.X, data.Y, h.weights, h.bias)
h

The perceptron should now converge and separate the data.

__QUESTION 1:__ Look at the previous two plots. What important difference between the two datasets can you see? Why do we need to run more epochs in order to converge? Try to be as precise as possible.

__ANSWER 1:__ 

## Interpreting perceptrons

Let's revisit the perceptron trained on `TwoFarGaussians`. The perceptron algorithm converges after 2 epochs.

In [ ]:
data = datasets.TwoFarGaussians
h = perceptron.Perceptron({'numEpoch': 2})
h.train(data.X, data.Y)
runClassifier.plotDataAndBoundary(data.X, data.Y, h.weights, h.bias)
h

The distance to the negative class looks uncomfortably small. The boundary gives an unfair share of space to the positive class, when it should be evenly divided. That's because the perceptron algorithm doesn't try to evenly divide the space - all it does is try to correctly classify the points.

Let's understand the consequences of training and deploying a model whose decision boundary is too close to one class, because this matters in the real world. In general, we expect that new data points will come from the same distribution as our training data, and a decision boundary that's "too close" to one class will make one kind of misclassification more often than another. In the case of the perceptron we just trained, if we give our model an input it hasn't seen before, and if our model outputs a bad prediction, it's more likely to be a false positive than a false negative. On some datasets, we might expect this to happen. But on this particular dataset, we know that our classes are very similar, and we shouldn't expect one mistake more than another. This is like a bug in the code! "Debugging" the algorithm will lead to a solution like the SVM, which tries to evenly divide the space between the classes. If we want to train a perceptron, then we would like to be able to _detect_ when our perceptron isn't doing what we expect, before we deploy the model, even if it scores perfect accuracy on the training data!

__TASK 3:__ We should be able to tell which class our perceptron is "biased" to predict by reading its bias parameter. But we can't do this for the perceptron above. Preprocess the data so that we recover the hidden interpretive power of the bias. Store the new data in `X`. (It's ok if the boundary moves, because of our initialization.)

In [ ]:
X = datasets.TwoFarGaussians.X
# your code here

Let's see if we can now interpret our model trained on `X`.

In [ ]:
h = perceptron.Perceptron({'numEpoch': 2})
h.train(X, data.Y)
runClassifier.plotDataAndBoundary(X, data.Y, h.weights, h.bias)
h

After preprocessing the data, the perceptron should again classify all points correctly, and will still be biased. But now we can look at the bias parameter and read the actual "bias" in classification.

__QUESTION 2:__ Why did we need to do this in order to interpret the bias?

__ANSWER 2:__ 

__QUESTION 3:__ Does the _magnitude_ of the bias tell us anything about how the perceptron misclassifies? Explain.

__ANSWER 3:__ 

## Perceptron for Sentiment Prediction

Let's continue to build our intuition for perceptron. We will apply the perceptron to sentiment prediction, to classify reviews in `SentimentData` as having positive sentiment (`+1`) or negative sentiment (`-1`). Now that our feature vectors are much larger, they are perhaps a bit harder to interpret. Let's first see how the perceptron performs.

__TASK 4:__ Using the tools provided in `runClassifier.py`, write code to generate a learning curve (x-axis=number of training examples) for the perceptron (5 epochs) on `SentimentData`.

In [ ]:
# your code here

__TASK 5:__ Now generate a plot of number of epochs versus train/test accuracy on the entire dataset.

In [ ]:
# your code here

As we learned in a previous task, it is good when we can interpret our model, so we can understand how they make their decisions. Let's see if we can interpret a perceptron trained on `SentimentData` and see how it classifies reviews. Our decision boundary will be in 8161-dimensional space, so it's not as easy to visualize the decision boundary (though you can learn more about methods for visualizing multi-dimensional decision boundaries). For now, we will interpret without visualization. Let's train.

In [ ]:
h = perceptron.Perceptron({'numEpoch': 8})
h.train(datasets.SentimentData.X, datasets.SentimentData.Y)
w,b = h.weights, h.bias
h

__TASK 6:__ Given `w` and `b`, find
- the 8 best predictors of positive sentiment
- the 8 best predictors of negative sentiment
- the 8 worst predictors of any sentiment

according to the perceptron.

_(Hint: weight at index_ `i` _corresponds to the word_ `datasets.SentimentData.words[i]`_)_

In [ ]:
# your code here

__QUESTION 4:__ Based on these groups of words, is the perceptron performing as we would expect it to? Are there any words that seem like they shouldn't be in the group? If so, then why are they there?

__ANSWER 4:__ 

## Learn more

### Bivariate Gaussian distribution
- [Multivariate normal distribution - Wikipedia](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)

### False positives and false negatives
- [Type I and type II errors - Wikipedia](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors)
- [Precision and recall - Wikipedia](https://en.wikipedia.org/wiki/Precision_and_recall)

### Visualizing multi-dimensional decision boundaries
- [Migut et al., 2015](https://staff.fnwi.uva.nl/m.a.migut/migut2015.pdf)
- [Rodrigues et al.](http://www.cs.rug.nl/~alext/PAPERS/SIBGRAPI18/paper2.pdf)
- [tmadl/highdimensional-decision-boundary-plot](https://github.com/tmadl/highdimensional-decision-boundary-plot)